## Final Project - Rafael Boccuni-Godfrey

All project code: [GitHub Repository](https://github.com/rboccunigodfrey/division-codebase)
STATUS: PROTOTYPE

### Overview
This project is meant to supplement my final project at Hampshire College, two robotic arrays (fretting hand and plucking hand) that can be mounted to a custom (but still made for a human) guitar to play it. The project for this class will be creating a simulation of how the solenoid array for the fretting hand would theoretically operate (travel times, non-instant movement, physical limitations), and then implement machine learning to  instruct a model to play MIDI songs by using the solenoids efficiently.

There are two components to this project:
1. Creating a Unity simulation and a python implementation of reading in MIDI files and converting them to a CSV representation that can be understood by the Unity program CSV reader, and includes all information needed for chords.
2. Implement machine learning techniques in aspects of the process
    - Velocity/note duration training
    - Reinforcement learning via a feedback mechanism from the simulation.

Here is a [video of randomized solenoid movement](https://youtu.be/W5E9KNPPcoI) and some screencaps showing the Unity side (and snippets of the C# code) of the project (you can also find this code [here](https://github.com/rboccunigodfrey/division-codebase/tree/master/unity/L_HAND/Assets/Scripts)):
![image](unityscreen1.png)
###### - Still of solenoid array
![image](unityscreen2.jpg)
###### - Implementation of solenoid movement and shrinkage as fret closeness increases
![image](unityscreen3.jpg)
###### - Integer-value control interface wrapper
#### Code Explanation:
Commands (where solenoid is a solenoid object, fret_hand is an array of solenoid objects, and arg is an integer):
SActions.TiltX.SetFor(solenoid, arg): arg = -1, 0, 1
SActions.TiltZ.SetFor(solenoid, arg): arg = -1, 0, 1
SActions.TrackPosition.SetFor(solenoid, arg): arg = 0-1 or 0-2
SActions.Activation.SetFor(solenoid, arg): arg = 0, 1
FHActions.TrackPosition.SetFor(fret_hand, arg): arg = 3-19
These commands can be used to control a fret hand object, and iterating through the 6 solenoids allows us to control them too.
Ideally, this simple interface with integer-only parameters will allow for simple I/O processing once I start generating python models.

### Progress Updates

#### Week of April 20th:

Progress:
- I used this week to focus on building a simulated environment in Unity to control a 2x3 solenoid array
- Wrote C# codebase for sending commands to the solenoids (can find in the repository link at the top of this document under unity/L_HAND/Assets/Scripts)
- Physical restraints programmed in (movement lerping, spacing):
https://youtu.be/W5E9KNPPcoI
- Takes into account how frets get more frequent as the hand goes up the neck
- Affects travel distance, solenoid track length, X and Z solenoid spacing

Challenges:
- The biggest challenges when writing this Unity program were a.) working with a language I am not quite familiar with (C#) and b.) formulating a way to update the relative positions of the solenoids to each other based on how close the frets around them are. When the solenoids travel along their paths, they need to know what the array position is, so they can travel to specific distances corresponding to the frets. Additionally, some fingers move positively along the neck, the other three move opposite, adding another level of complexity as the reverse-traveling solenoids must travel farther distances than the forward-traveling ones. Making sure the solenoids could both set and get their positions based on these constraints was the largest conceptual challenge this week.


#### Week of April 27th:

Progress:
- Began writing Python code to generate chords based on some basic music theory. The program generates a chord pattern in the form of [x_1, x_2, x_3, x_4, x_5, x_6] where x_n is between -1 and 5 inclusive and corresponds to the fretted position on string n. E.g. [0, 3, 3, 2, 0, 0] corresponds to an open E chord, [-1, 0, 2, 2, 2, 0] to open A, etc. These numbers can be shifted by the value corresponding to the fret position the hand is located:


Challenges:
- The biggest challenge for me this week was thinking about the best way to represent notes in the code. I have a background in music, so it was interesting trying to think of how to learn music from a computer’s perspective. I decided that for now, the easiest representation makes the most sense: a length 6 integer array to create a chord at given fret positions, alongside duration and velocity information
- I did easy parts of this chapter this week, the more difficult task will be coming up with a mechanism to translate this chord notation into machine commands of the types outlined in the unity code. The challenge arises because the program needs to learn how the robot’s physical (or in the simulation’s case, simulated) limitations prevent certain actions from being performed. In other words, it needs to learn to optimize its actions.
- This was the main reason I wanted to make a simulation: I need some sort of reinforcement algorithm to use the state of the simulation to gauge the success of attempted actions, so having a local simulation running that can easily provide those numbers (albeit estimated) is a lot easier than building the whole circuit out.


#### What's left?
- Establish some feedback mechanism between the c# code and python code (i.e. unity writing solenoid position out every keyframe so that python can compare the desired position vs the actual position to get a rating of how well the command worked.
- Successfully convert python csv output to solenoids. Currently, I am doing this by just converting the midi to csv in python, then figuring out how the solenoids should mobe in the c# code. However, I will likely change that when I implement the feedback above, since it will be easier to do all the training for the solenoid positioning in python. Therefore, an output file would not include chord info as it currently does, but keyframed per-solenoid movement mapping instead.
- Visualize learning process via graphs, histograms, etc.

### Jupyter Document

The first part of this Jupyter document focuses on the Python side of the first component. A [MIDI file](twinkle-twinkle-little-star.mid) is imported, the `mido` library is used to extract event information from the file (on/off, velocity, note, and time). Several helper functions are used to convert individual event's notes into chord info (i.e. if two notes or more notes are playing at one time, they are grouped together). Finally, this chord information is generated for the whole MIDI file on a per-keyframe basis, and exported to a [CSV file](twinkle.csv). Each note in each chord contains info on the time it was activated and the time it was queried.

In [2]:
# Imports

import random
import numpy as np
from enum import Enum
from sklearn.ensemble import RandomForestRegressor
import mido

In [3]:
mid = mido.MidiFile("twinkle-twinkle-little-star.mid")
mid

MidiFile(type=1, ticks_per_beat=256, tracks=[
  MidiTrack([
    MetaMessage('time_signature', numerator=4, denominator=4, clocks_per_click=24, notated_32nd_notes_per_beat=8, time=0),
    MetaMessage('key_signature', key='C', time=0),
    MetaMessage('set_tempo', tempo=631577, time=0),
    MetaMessage('track_name', name='Greensleeves', time=0),
    MetaMessage('text', text='Traditional', time=0),
    MetaMessage('copyright', text='Jim Paterson', time=0),
    MetaMessage('end_of_track', time=1)]),
  MidiTrack([
    Message('program_change', channel=0, program=0, time=0),
    Message('control_change', channel=0, control=121, value=0, time=0),
    Message('control_change', channel=0, control=64, value=0, time=0),
    Message('control_change', channel=0, control=10, value=63, time=0),
    Message('control_change', channel=0, control=7, value=95, time=0),
    Message('note_on', channel=0, note=60, velocity=77, time=0),
    Message('note_on', channel=0, note=52, velocity=63, time=0),
    Mess

In [4]:
data = []
# msg is type mido.MetaMessage
prevTime = 0
for msg in mid:
    if msg.type == "note_on" or msg.type == "note_off":
        calc_time = round(msg.time * (487/1.2014765585))
        data.append({
            "activated": int(msg.type == "note_on"),
            "velocity": msg.bytes()[-1],
            "note": msg.bytes()[1],
            "time": calc_time + prevTime})
        prevTime += calc_time
data

[{'activated': 1, 'velocity': 77, 'note': 60, 'time': 0},
 {'activated': 1, 'velocity': 63, 'note': 52, 'time': 0},
 {'activated': 1, 'velocity': 64, 'note': 48, 'time': 0},
 {'activated': 0, 'velocity': 0, 'note': 60, 'time': 244},
 {'activated': 1, 'velocity': 83, 'note': 60, 'time': 256},
 {'activated': 0, 'velocity': 0, 'note': 52, 'time': 487},
 {'activated': 0, 'velocity': 0, 'note': 48, 'time': 487},
 {'activated': 0, 'velocity': 0, 'note': 60, 'time': 500},
 {'activated': 1, 'velocity': 90, 'note': 67, 'time': 512},
 {'activated': 1, 'velocity': 77, 'note': 60, 'time': 512},
 {'activated': 1, 'velocity': 71, 'note': 48, 'time': 512},
 {'activated': 0, 'velocity': 0, 'note': 67, 'time': 756},
 {'activated': 1, 'velocity': 87, 'note': 67, 'time': 768},
 {'activated': 0, 'velocity': 0, 'note': 60, 'time': 999},
 {'activated': 0, 'velocity': 0, 'note': 48, 'time': 999},
 {'activated': 0, 'velocity': 0, 'note': 67, 'time': 1012},
 {'activated': 1, 'velocity': 86, 'note': 69, 'time':

In [5]:
open_string_values = [40, 45, 50, 55, 59, 64]

def get_fret_positions(note):
    positions = []
    for string_value in open_string_values:
        if note["note"] >= string_value:
            if (note["note"]-string_value) > 22:
                positions.append(-1)
            else:
                positions.append(note["note"] - string_value)
        else:
            positions.append(-1)
    return positions

In [6]:
def get_notes_at_time(notes_data, at_time):
    # detect which notes are "activated" at a given time
    notes = {}
    for note in notes_data:
        if note["time"] <= at_time:
            if note["activated"]:
                notes[note["note"]] = note
            else:
                if note["note"] in list(notes.keys()):
                    notes.pop(note["note"])
    return list(notes.values())

In [7]:
def get_chord_at_time(note_data, at_time):
    fret_positions_velocity = []
    for note in get_notes_at_time(note_data, at_time):
        fret_positions_velocity.append([get_fret_positions(note), note["velocity"], note["time"], at_time])
    strings = []
    chord = np.full((6, 4), -1)
    for note_i in range(6):
        chord[note_i, -1] = at_time
    for position in fret_positions_velocity:
        # get the lowest not -1 fret position and index of lowest fret position in each position list
        lowest = min([x for x in position[0] if x != -1 and position[0].index(x) not in strings])
        strings.append(position[0].index(lowest))
        chord[position[0].index(lowest)] = [lowest, position[1], position[2], position[3]]

    return chord

get_chord_at_time(data, 600)

array([[ -1,  -1,  -1, 600],
       [  3,  71, 512, 600],
       [ -1,  -1,  -1, 600],
       [ -1,  -1,  -1, 600],
       [  1,  77, 512, 600],
       [  3,  90, 512, 600]])

In [8]:
def get_keyframes(note_data):
    keyframes = set()
    for note in note_data:
        keyframes.add(note["time"])

    return list(sorted(keyframes))

#find smallest distance between keyframes
def get_smallest_distance(keyframes):
    smallest = 100000
    for key_i in range(len(keyframes)-1):
        if keyframes[key_i+1] - keyframes[key_i] < smallest:
            smallest = keyframes[key_i+1] - keyframes[key_i]
    return smallest

chords = np.full((len(get_keyframes(data)), 6, 4), -1)
for k_i, keyframe in enumerate(get_keyframes(data)):
    chords[k_i] = get_chord_at_time(data, keyframe)

chords

array([[[   -1,    -1,    -1,     0],
        [    3,    64,     0,     0],
        [    2,    63,     0,     0],
        [   -1,    -1,    -1,     0],
        [    1,    77,     0,     0],
        [   -1,    -1,    -1,     0]],

       [[   -1,    -1,    -1,   244],
        [    3,    64,     0,   244],
        [    2,    63,     0,   244],
        [   -1,    -1,    -1,   244],
        [   -1,    -1,    -1,   244],
        [   -1,    -1,    -1,   244]],

       [[   -1,    -1,    -1,   256],
        [    3,    64,     0,   256],
        [    2,    63,     0,   256],
        [   -1,    -1,    -1,   256],
        [    1,    83,   256,   256],
        [   -1,    -1,    -1,   256]],

       ...,

       [[   -1,    -1,    -1, 11764],
        [   -1,    -1,    -1, 11764],
        [   -1,    -1,    -1, 11764],
        [   -1,    -1,    -1, 11764],
        [   -1,    -1,    -1, 11764],
        [   -1,    -1,    -1, 11764]],

       [[   -1,    -1,    -1, 11776],
        [    3,    74, 11776,

In [64]:
instructions = np.empty((len(chords), 5, 6), dtype=int)

for chord_i, chord in enumerate(chords):
    lowest_fret = -1
    highest_fret = -1
    chord_info = np.full((6, 5), -1)
    for note in chord:
        if note[0] != -1:
            if note[0] < lowest_fret or lowest_fret == -1:
                lowest_fret = note[0]
            if note[0] > highest_fret or highest_fret == -1:
                highest_fret = note[0]
    if lowest_fret == -1:
        chord_info = np.full((5, 6), -1)
        chord_info[0, 1] = chord[0][3]

    else:
        solenoid_moves = [100, 100, 100, 100, 100, 100]
        solenoid_tilts = [0, 0, 0, 0, 0, 0]
        solenoid_velocities = [-1, -1, -1, -1, -1, -1]
        for note_i, note in enumerate(chord):
            if note[0] >= 0:
                if note_i in [0, 2, 4]:  # Row A solenoids
                    move = note[0] - (lowest_fret + 2)
                    if move >= 0:
                        solenoid_moves[note_i] = min(move, 2)  # Move a maximum of 1 fret back from origin
                        solenoid_tilts[note_i] = 0
                        solenoid_velocities[note_i] = note[1]
                    else:
                        # Assign the task to the nearest available solenoid in row B
                        if note_i < 5:
                            nearest_row_b = note_i + 1
                            solenoid_tilts[note_i] = -1
                            if solenoid_moves[nearest_row_b] != 100 and note_i > 0:
                                nearest_row_b = note_i - 1
                                solenoid_tilts[note_i] = 1
                        else:
                            nearest_row_b = note_i - 1
                            solenoid_tilts[note_i] = 1
                        row_b_move = -move  # Move a maximum of 2 frets forward from origin
                        solenoid_moves[nearest_row_b] = row_b_move
                        solenoid_velocities[nearest_row_b] = note[1]

                else:  # Row B solenoids
                    move = note[0] - (lowest_fret + 2)
                    if move >= 0:
                        solenoid_moves[note_i] = min(move, 2)  # Move a maximum of 1 fret back from origin
                        solenoid_tilts[note_i] = 0
                        solenoid_velocities[note_i] = note[1]
                    else:
                        # Assign the task to the nearest available solenoid in row B
                        if note_i < 5:
                            nearest_row_a = note_i + 1
                            solenoid_tilts[note_i] = 1
                            if solenoid_moves[nearest_row_a] != 100 and note_i > 0:
                                nearest_row_a = note_i - 1
                                solenoid_tilts[note_i] = -1
                        else:
                            nearest_row_a = note_i - 1
                            solenoid_tilts[note_i] = -1
                        row_a_move = -move  # Move a maximum of 2 frets forward from origin
                        solenoid_moves[nearest_row_a] = row_a_move
                        solenoid_velocities[nearest_row_a] = note[1]
        chord_info = np.array([
                         [lowest_fret + 2, chord[0][3], -1, -1, -1, -1],
                         [-1 if x == 100 else x for x in solenoid_moves],
                         solenoid_tilts,
                         solenoid_velocities,
                         [note[2] for note in chord]])
    instructions[chord_i] = chord_info

instructions

array([[[    3,     0,    -1,    -1,    -1,    -1],
        [   -1,     0,    -1,     1,    -1,     2],
        [    0,     0,    -1,     0,    -1,     0],
        [   -1,    64,    -1,    63,    -1,    77],
        [   -1,     0,     0,    -1,     0,    -1]],

       [[    4,   244,    -1,    -1,    -1,    -1],
        [   -1,    -1,     1,     2,    -1,    -1],
        [    0,     1,    -1,     0,     0,     0],
        [   -1,    -1,    64,    63,    -1,    -1],
        [   -1,     0,     0,    -1,    -1,    -1]],

       [[    3,   256,    -1,    -1,    -1,    -1],
        [   -1,     0,    -1,     1,    -1,     2],
        [    0,     0,    -1,     0,    -1,     0],
        [   -1,    64,    -1,    63,    -1,    83],
        [   -1,     0,     0,    -1,   256,    -1]],

       ...,

       [[   -1, 11764,    -1,    -1,    -1,    -1],
        [   -1,    -1,    -1,    -1,    -1,    -1],
        [   -1,    -1,    -1,    -1,    -1,    -1],
        [   -1,    -1,    -1,    -1,    -1,  

In [65]:
# Save 3D instruction array to 2D CSV
inst2d = instructions.reshape((instructions.shape[0], -1))
np.savetxt("twinkle.csv", inst2d, delimiter=",", fmt="%d")
inst2d

array([[    3,     0,    -1, ...,    -1,     0,    -1],
       [    4,   244,    -1, ...,    -1,    -1,    -1],
       [    3,   256,    -1, ...,    -1,   256,    -1],
       ...,
       [   -1, 11764,    -1, ...,    -1,    -1,    -1],
       [    3, 11776,    -1, ...,    -1, 11776,    -1],
       [   -1, 12263,    -1, ...,    -1,    -1,    -1]])

### Generating Chords

The code below implements some concepts in music theory that allows all fret positions for all key/scale pairs to be calculated using a simple list rearrangement. This allows random note sequences and chords to be generated. I may be able to use genetic methods to improve the quality of autogenerated songs for use by the simulation.

#### Code explanation:
PATTERNS_C_IONIAN contains all notes across all strings in 12 positions that fit into the c major scale. This list can be shifted (i.e. position shifted) by an offset defined by the desired key (“C”, “C#”...) and scale (Scales.IONIAN, Scales.DORIAN, etc) to get a pattern list that works for any key and scale. This is done in get_pattern. Generate_chord creates a random chord in a given key/scale/pattern.

In [ ]:
NOTES = ["C", "C#", "D", "D#", "E", "F", "F#", "G", "G#", "A", "A#", "B"]


class Scales(Enum):
    IONIAN = 0
    DORIAN = 2
    PHRYGIAN = 4
    LYDIAN = 5
    MIXOLYDIAN = 7
    AEOLIAN = 9
    LOCRIAN = 11


PATTERNS_C_IONIAN = [
    [[0, 1, 3], [0, 2, 3], [0, 2, 3], [0, 2, 4], [0, 1, 3], [0, 1, 3]],
    [[0, 2, 4], [1, 2, 4], [1, 2, 4], [1, 3, 4], [0, 2, 4], [0, 2, 4]],
    [[1, 3], [0, 1, 3], [0, 1, 3], [0, 2, 3], [1, 3, 4], [1, 3]],
    [[0, 2, 4], [0, 2, 4], [0, 2, 4], [1, 2, 4], [0, 2, 3], [0, 2, 4]],
    [[1, 3, 4], [1, 3, 4], [1, 3], [0, 1, 3], [2, 3, 4], [1, 3, 4]],
    [[0, 2, 3], [0, 2, 3], [0, 2, 4], [0, 2, 4], [0, 1, 3], [0, 2, 3]],
    [[1, 2, 4], [1, 2, 4], [1, 3, 4], [1, 3, 4], [0, 2, 4], [1, 2, 4]],
    [[0, 1, 3], [0, 1, 3], [0, 2, 3], [0, 2, 3], [1, 3], [0, 1, 3]],
    [[0, 2, 4], [0, 2, 4], [1, 2, 4], [1, 2, 4], [0, 2, 4], [0, 2, 4]],
    [[1, 3, 4], [1, 3], [0, 1, 3], [0, 1, 3], [1, 3, 4], [1, 3, 4]],
    [[0, 2, 3], [0, 2, 4], [0, 2, 4], [0, 2, 4], [0, 2, 3], [0, 2, 3]],
    [[1, 2, 4], [1, 3, 4], [1, 3, 4], [1, 3], [1, 2, 4], [1, 2, 4]]]


def get_pattern(key, scale=Scales.IONIAN):
    to_index = (len(NOTES) - NOTES.index(key)) + scale.value
    if to_index >= len(PATTERNS_C_IONIAN):
        to_index = to_index - len(PATTERNS_C_IONIAN)

    return PATTERNS_C_IONIAN[to_index:] + PATTERNS_C_IONIAN[:to_index]


def generate_chord(pattern=None, key="C", scale=Scales.IONIAN, position=0, include_position=False, note_count=3):
    # Generate a random chord
    if pattern is None:
        pattern = get_pattern(key, scale=scale)
    chord = []
    chosen_strings = random.sample(range(6), note_count)
    for i in range(6):
        if i in chosen_strings:
            chord.append(random.choice(pattern[position][i]) + include_position * position)
        else:
            chord.append(-1)
    return chord


def generate_note(pattern=None, key="C", scale=Scales.IONIAN, position=0, include_position=False):
    generate_chord(pattern=pattern, key=key, scale=scale, position=position, include_position=include_position,
                   note_count=1)


get_pattern("C", scale=Scales.DORIAN)

### Using SKLearn to predict velocities and durations

The code below contains implementations of a random forest regressor currently only trained on randomly generated training chords and velocity/duration. It demonstrates that a sensible value for a velocity and duration can be found using random forest. This is preliminary though, and there is no indication that training these values will get sensible outputs in the context of being at a certain position in a song. I might have to approach this a different way.

In [10]:
def post_process_duration(y_pred):
    # Round duration to the nearest multiple of 0.125
    y_pred[:] = np.round(y_pred[:] / 0.125) * 0.125

    return y_pred

In [11]:
def post_process_velocity(y_pred):
    # Clip velocity to the range 1-127
    y_pred[:] = np.clip(np.round(y_pred[:]), 1, 127)

    return y_pred

In [12]:
def preprocess_input(x):
    # Convert chords to binary
    x = np.array([[''.join([bin(x)[2:].zfill(6) for x in row]) for row in level] for level in (x + 1)])

    return x

In [13]:
POPULATION = 100
NOTES_PER = 100

In [14]:
rand_chords_x = np.array([[generate_chord(position=i % 12, note_count=(random.randrange(1, 5)), include_position=True) for i in range(NOTES_PER)] for _ in range(POPULATION)])
rand_chords_x

array([[[ 3,  3,  0,  0, -1, -1],
        [-1, -1,  2,  5,  3,  3],
        [-1,  3, -1, -1, -1,  5],
        ...,
        [-1, -1, -1, -1, -1,  1],
        [-1,  5,  3, -1, -1, -1],
        [-1, -1, -1, -1,  3, -1]],

       [[-1, -1,  2, -1,  3, -1],
        [-1,  3, -1,  5,  1,  1],
        [-1,  3,  5, -1,  5, -1],
        ...,
        [-1, -1, -1,  4, -1,  5],
        [ 5,  3,  3,  5, -1, -1],
        [ 3,  3, -1, -1, -1, -1]],

       [[-1, -1, -1, -1,  0, -1],
        [ 1,  2,  5,  4, -1, -1],
        [-1,  2, -1, -1,  6, -1],
        ...,
        [-1, -1, -1, -1,  3, -1],
        [-1,  3,  5,  2, -1,  5],
        [-1, -1, -1, -1, -1,  7]],

       ...,

       [[ 3,  0, -1,  0,  1, -1],
        [ 5, -1, -1,  5, -1,  5],
        [ 3,  3, -1, -1,  3, -1],
        ...,
        [-1,  3,  5,  5,  3, -1],
        [-1,  2, -1, -1, -1, -1],
        [-1, -1,  5,  5, -1, -1]],

       [[-1, -1, -1,  0, -1, -1],
        [ 1, -1, -1, -1, -1, -1],
        [-1,  5,  3,  4,  3, -1],
        .

In [15]:
rand_chords_x_flat = preprocess_input(rand_chords_x)
rand_chords_x_flat

array([['000100000100000001000001000000000000',
        '000000000000000011000110000100000100',
        '000000000100000000000000000000000110', ...,
        '000000000000000000000000000000000010',
        '000000000110000100000000000000000000',
        '000000000000000000000000000100000000'],
       ['000000000000000011000000000100000000',
        '000000000100000000000110000010000010',
        '000000000100000110000000000110000000', ...,
        '000000000000000000000101000000000110',
        '000110000100000100000110000000000000',
        '000100000100000000000000000000000000'],
       ['000000000000000000000000000001000000',
        '000010000011000110000101000000000000',
        '000000000011000000000000000111000000', ...,
        '000000000000000000000000000100000000',
        '000000000100000110000011000000000110',
        '000000000000000000000000000000001000'],
       ...,
       ['000100000001000000000001000010000000',
        '000110000000000000000110000000000110',
        '0

In [16]:
rand_durations = np.array(
    [[round(random.random() / 0.125) * 0.125 for _ in range(NOTES_PER)] for _ in range(POPULATION)])
rand_durations

array([[0.125, 0.125, 0.   , ..., 1.   , 0.125, 0.25 ],
       [0.25 , 0.375, 0.5  , ..., 0.875, 0.   , 0.5  ],
       [0.5  , 0.125, 0.375, ..., 0.   , 0.125, 0.875],
       ...,
       [1.   , 0.125, 0.75 , ..., 0.25 , 0.375, 0.   ],
       [0.5  , 0.875, 0.125, ..., 0.125, 0.125, 0.5  ],
       [0.75 , 0.625, 1.   , ..., 0.625, 0.75 , 0.875]])

In [17]:
rand_velocities = np.array([[random.randrange(50, 127) for _ in range(NOTES_PER)] for _ in range(POPULATION)])

rand_velocities

array([[ 53,  78,  84, ...,  70, 102,  62],
       [ 74,  79,  88, ...,  90,  52, 102],
       [ 92, 117,  78, ..., 104,  75,  56],
       ...,
       [107, 115, 122, ...,  65,  89, 120],
       [ 83,  81,  83, ...,  96, 103, 117],
       [112,  99,  77, ...,  89,  80, 108]])

In [18]:
dur_regr = RandomForestRegressor(n_estimators=100)
dur_regr.fit(rand_chords_x_flat, rand_durations)
vel_regr = RandomForestRegressor(n_estimators=100)
vel_regr.fit(rand_chords_x_flat, rand_velocities)

X_test = np.array([[generate_chord(position=i % 12, note_count=(random.randrange(1, 5)), include_position=True) for i in
                    range(NOTES_PER)] for _ in range(POPULATION)])
preprocess_input(X_test)

array([['000100000001000001000000000000000000',
        '000110000110000110000000000000000110',
        '000000000000000000000000000000000110', ...,
        '000110000011000000000000000000000000',
        '000000000110000000000000000111000000',
        '000100000000000110000000000111001000'],
       ['000000000000000000000011000001000100',
        '000100000110000000000000000100000100',
        '000000000000000000000110000000000000', ...,
        '000000000000000000000110000010000000',
        '000000000000000000000110000000000000',
        '001000000100000110000000000000000000'],
       ['000000000100000000000011000000000000',
        '000000000000000110000000000010000010',
        '000110000110000100000101000000000000', ...,
        '000100000011000110000000000000000000',
        '000000000100000000000000000000000000',
        '000000000100000110000000000110000000'],
       ...,
       ['000000000001000000000000000100000000',
        '000000000000000000000011000000000000',
        '0

In [19]:
post_process_duration(dur_regr.predict(preprocess_input(X_test)))

array([[0.375, 0.5  , 0.5  , ..., 0.5  , 0.5  , 0.5  ],
       [0.625, 0.5  , 0.5  , ..., 0.5  , 0.5  , 0.5  ],
       [0.5  , 0.5  , 0.625, ..., 0.625, 0.5  , 0.5  ],
       ...,
       [0.375, 0.5  , 0.625, ..., 0.375, 0.5  , 0.375],
       [0.5  , 0.5  , 0.5  , ..., 0.625, 0.5  , 0.375],
       [0.625, 0.5  , 0.5  , ..., 0.375, 0.5  , 0.5  ]])

In [20]:
post_process_velocity(vel_regr.predict(preprocess_input(X_test)))

array([[88., 87., 87., ..., 84., 95., 86.],
       [92., 88., 83., ..., 84., 92., 90.],
       [84., 88., 87., ..., 87., 93., 83.],
       ...,
       [91., 82., 87., ..., 88., 87., 91.],
       [88., 94., 86., ..., 83., 88., 85.],
       [89., 89., 82., ..., 85., 94., 87.]])